<a href="https://colab.research.google.com/github/Ihoold/pixelgen/blob/master/funit4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from unittest import mock
import os
import numpy as np
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from tensorflow.python.estimator import estimator
from google.colab import drive, auth

In [0]:
R, C = 4, 3
EXAMPLES = R * C
    
CHANNELS = 4
LATENT_DIM = 128
ADD_NOISE_TO_EXAMPLE = False

DROPOUT_PROB = 0.5
ALPHA = 0.2
BATCH_SIZE = 1024
EPOCHS = 100000
EVAL_EPOCHS = 1000
G_LR = 0.0002
D_LR = 0.0001
KERNEL_SIZE = 4
NUM_CLASSES = 21

MODEL_NAME = 'FUNIT'
RUN_NAME = 'FUNIT_only_gen_13'

data_train_file = 'gs://tputestingmnist/datasets/dataset_train_funit.tfrecords'
data_test_file = 'gs://tputestingmnist/datasets/dataset_test_funit.tfrecords'
MODEL_DIR = 'gs://tputestingmnist/{}/{}/'.format(MODEL_NAME, RUN_NAME)
# MODEL_DIR = 'gs://tputestingmnist/{}/'.format(RUN_NAME)
GOOGLE_DRIVE_DIR = '/content/gdrive/My Drive/Programowanie/PixelGen/{}'.format(RUN_NAME)
TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])

In [0]:
#################################### SETUP #####################################

def setup():
    drive.mount('/content/gdrive')
    auth.authenticate_user()


def upload_credentials():
    # Upload credentials to TPU.
    with tf.Session(TF_MASTER) as sess:    
        with open('/content/adc.json', 'r') as f:
            auth_info = json.load(f)
        tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)

In [0]:
################################# DATA INPUT ###################################

def parser(serialized_example):
        """Parses a single example into image and label tensors."""
        features = tf.parse_single_example(
            serialized_example,
            features={
                'image_content': tf.FixedLenFeature([], tf.string),
                'image_style': tf.FixedLenFeature([], tf.string),
                'label': tf.FixedLenFeature([], tf.int64) 
            })

        content_image = tf.decode_raw(features['image_content'], tf.uint8)
        content_image.set_shape([48 * 48 * 4])
        content_image = tf.reshape(content_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        content_image = tf.cast(content_image, dtype=tf.float32) / 127.5 - 1

        style_image = tf.decode_raw(features['image_style'], tf.uint8)
        style_image.set_shape([48 * 48 * 4])
        style_image = tf.reshape(style_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        style_image = tf.cast(style_image, dtype=tf.float32) / 127.5 - 1
        
        label = features['label']

        return {'style_images': style_image,
                'content_images': content_image}, label


def make_input_fn(is_training=True, one_batch=False):
    def input_fn(params):
        batch_size = params['batch_size']
        dataset = tf.data.TFRecordDataset(data_train_file, buffer_size=8*1024*1024)
        dataset = dataset.map(parser).cache().shuffle(batch_size)
        if is_training:
            dataset = dataset.repeat()
        if one_batch:
            dataset = dataset.take(params['batch_size'])
            
        dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(8)

        return dataset
    return input_fn


def predict_input_fn(params):
    batch_size = params['batch_size']
    dataset = tf.data.TFRecordDataset(data_test_file).map(parser).cache().shuffle(batch_size)
    style_images, content_images, _ = dataset.prefetch(batch_size).batch(batch_size, drop_remainder=True).make_one_shot_iterator().get_next()
    
    features = {
        'style_images': style_images,
        'content_images': content_images,
    }
    return features, None


In [0]:
############################### DATA SAVEING ###################################
 
def images_to_zero_one(images):
        return np.clip(np.array(images) * 0.5 + 0.5, 0., 1.)


def save_imgs(epoch, images):
    if not os.path.exists(GOOGLE_DRIVE_DIR):
        os.mkdir(GOOGLE_DRIVE_DIR)

    # Rescale images to 0 - 1
    images = images_to_zero_one(images)
    fig, axs = plt.subplots(R, C, figsize=(20,20))

    for i in range(R):
        for j in range(C):
            axs[i,j].imshow(images[C*i + j])
            axs[i,j].axis('off')

    fig.savefig(os.path.join(GOOGLE_DRIVE_DIR, '{}.png'.format(epoch)))
    plt.close()

In [0]:
################################## MODEL #######################################

def _relu(x):
    return tf.nn.relu(x)


def _leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=ALPHA)


def _instance_norm(x):
    return tf.contrib.layers.instance_norm(x)


def _make_adain_norm(beta, gamma):
    def _adain_norm(x):
        mean, var = tf.nn.moments(class_latent, [1, 2], keep_dims=True)
        outputs = tf.nn.batch_normalization(inputs, mean, var, beta, gamma, epsilon=1e-5, name='in_bn_{}'.format(index))
        return outputs
    return _adain_norm


def _dense(x, neurons, name, activation=None):
    return tf.layers.dense(x, neurons, name=name, activation=activation,
                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def _conv2d(x, filters, kernel_size, stride, name, activation=None):
    return tf.layers.conv2d(x, filters, [kernel_size, kernel_size], 
                            strides=[stride, stride], activation=activation,
                            padding='same', name=name,
                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

def _pooling(x, kernel, stride, index):
    return tf.layers.average_pooling2d(x, pool_size=kernel, strides=stride, name='pool_{}'.format(index))

def _deconv2d(x, filters, stride, name, activation=None):
    return tf.layers.conv2d_transpose(x, filters, [KERNEL_SIZE, KERNEL_SIZE],
                                      strides=[stride, stride], activation=activation,
                                      padding='same', name=name,
                                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def convolution_block(x, filters, kernel_size, resize_factor, index, activation=_relu, activation_first=False, normalization=None):
    if activation and activation_first:
        x = activation(x)
    x = _conv2d(x, kernel_size=kernel_size, filters=filters, stride=resize_factor, activation=None, name='conv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation and not activation_first:
        x = activation(x)
    return x


def deconvolution_block(x, filters, resize_factor, index, activation=_relu, normalization=None):
    x = _deconv2d(x, filters=filters, stride=resize_factor, activation=None, name='deconv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def fully_connected_block(x, neurons, index, activation=_relu, normalization=None):
    x = _dense(x, neurons=neurons, activation=None, name='fc_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def residual_block(x, filters_in, filters_out, index, kernel_size=3, activation=_relu, activation_first=False, normalization=None):
    forwarded_x = convolution_block(x, filters_in, kernel_size, resize_factor=1, 
                                    index='res_con1_{}'.format(index), 
                                    activation_first=activation_first, activation=activation)
    
    forwarded_x = convolution_block(forwarded_x, filters_out, kernel_size, resize_factor=1, 
                                    index='res_con2_{}'.format(index), 
                                    activation_first=activation_first, activation=None)
    
    if filters_in != filters_out:
        x = convolution_block(x, filters_out, kernel_size=1, resize_factor=1, index='res_con3_{}'.format(index), activation=None)
    x = tf.add(forwarded_x, x)
    return x


def adain_residual_block(x, beta, gamma, filters_in, filters_out, index, activation=_relu, activation_first=False):
    with tf.variable_scope('adain_{}'.format(index)):
        normalization = _make_adain_norm(beta, gamma)
        return residual_block(x, filters_in, filters_out, index, 3, activation, activation_first, normalization)


def content_encoder(x):
    x = convolution_block(x, 64, kernel_size=7, resize_factor=1, normalization=_instance_norm, index='con1')
    x = convolution_block(x, 128, kernel_size=4, resize_factor=2, normalization=_instance_norm, index='con2')
    x = convolution_block(x, 256, kernel_size=4, resize_factor=2, normalization=_instance_norm, index='con3')
    x = convolution_block(x, 512, kernel_size=4, resize_factor=2, normalization=_instance_norm, index='con4')
    
    x = residual_block(x, 512, 512, index='res1', normalization=_instance_norm)
    x = residual_block(x, 512, 512, index='res2', normalization=_instance_norm)
    return x


def class_encoder(x):
    x = convolution_block(x, 64, kernel_size=7, resize_factor=1, index='cls1')
    x = convolution_block(x, 128, kernel_size=4, resize_factor=2, index='cls2')
    x = convolution_block(x, 256, kernel_size=4, resize_factor=2, index='cls3')
    x = convolution_block(x, 512, kernel_size=4, resize_factor=2, index='cls4')
    x = convolution_block(x, 1024, kernel_size=4, resize_factor=2, index='cls5')
    x = _pooling(x, 3, 3, index='cls6')
    return x


def mlp(x, feature_number):
    x = fully_connected_block(x, 256, 'mlp1')
    x = fully_connected_block(x, 256, 'mlp2')
    x = fully_connected_block(x, feature_number, 'mlp3', activation=None)
    
    return x


def split_into_four_variable_sets(class_vars, num_features):
    b1 = class_vars[:, :num_features]
    g1 = class_vars[:, num_features:2*num_features]
    b2 = class_vars[:, 2*num_features:3*num_features]
    g2 = class_vars[:, 3*num_features:]
    return b1, g1, b2, g2
    

def decoder(content_latent, class_latent):
    # class code
    encoder_features_num = content_latent.shape[3]
    class_vars = mlp(class_latent, 4*encoder_features_num)
    b1, g1, b2, g2 = split_into_four_variable_sets(class_vars, encoder_features_num)
   
    # ADAIN
    x = adain_residual_block(content_latent, b1, g1, 512, 512, index='res3')
    x = adain_residual_block(x, b2, g2, 512, 512, index='res4')

    # Scale up
    x = deconvolution_block(x, 512, resize_factor=2, normalization=_instance_norm, index='dec0')
    x = deconvolution_block(x, 256, resize_factor=2, normalization=_instance_norm, index='dec1')
    x = deconvolution_block(x, 128, resize_factor=2, normalization=_instance_norm, index='dec2')
    x = deconvolution_block(x, CHANNELS, resize_factor=1, normalization=None, index='dec4', activation=tf.tanh)
    
    return x


class Funit:

    @staticmethod
    def discriminator(x, label, scope='Discriminator'):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            x = convolution_block(x, 64, kernel_size=7, resize_factor=1, index='disc_11')
            
            x = residual_block(x, 64, 64, 'disc_12', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 64, 128, 'disc_13', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_14')
            
            x = residual_block(x, 128, 128, 'disc_15', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 128, 256, 'disc_16', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_17')
            
            x = residual_block(x, 256, 256, 'disc_18', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 256, 512, 'disc_19', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_20')
            
            x = residual_block(x, 512, 512, 'disc_21', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 512, 1024, 'disc_22', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_23')
            
            x = residual_block(x, 1024, 1024, 'disc_24', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 1024, 1024, 'disc_25', kernel_size=3, activation=_leaky_relu, activation_first=True)
            
            x = _leaky_relu(x)
            x = tf.layers.flatten(x)
            x = _dense(x, NUM_CLASSES, 'disc_out')
            label = tf.expand_dims(label, axis=1)
            x = tf.gather_nd(x, label, batch_dims=1)
            
            return x
          
    @staticmethod
    def generator(image_content, image_style, is_training, scope='Generator'):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            class_latent = class_encoder(image_content)
            content_latent = content_encoder(image_style)
            x = decoder(content_latent, class_latent)
            return x

In [0]:
################################ MODEL FUN #####################################
def make_model_fn(model):

    def model_fn(features, labels, mode, params):
        with tf.variable_scope('inputs'):
            content_images = features['content_images']
            style_images = features['style_images']
        generated_images = model.generator(content_images, style_images, (mode == tf.estimator.ModeKeys.TRAIN))
        
        # PREDICT #
        if mode == tf.estimator.ModeKeys.PREDICT:
            predictions = {
                'generated_images': generated_images,
                'content_images': content_images,
                'style_images': style_images
            }
            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, predictions=predictions)

        # Discriminator loss
        d_on_data_logits = tf.squeeze(model.discriminator(style_images, labels))
        d_on_g_logits = tf.squeeze(model.discriminator(generated_images, labels))

        
        with tf.variable_scope('losses'):
            d_loss = tf.contrib.gan.losses.wargs.modified_discriminator_loss(
                discriminator_real_outputs=d_on_data_logits,
                discriminator_gen_outputs=d_on_g_logits,
                reduction=tf.losses.Reduction.NONE,
                label_smoothing=0.2
            )
            d_loss_reduced = tf.reduce_mean(d_loss)

            # Generator loss
            g_loss = tf.contrib.gan.losses.wargs.modified_generator_loss(
                discriminator_gen_outputs=d_on_g_logits,
                reduction=tf.losses.Reduction.NONE
            )
            g_loss_reduced = tf.reduce_mean(g_loss)
            
        # TRAIN #
        if mode == tf.estimator.ModeKeys.TRAIN:
            
            with tf.variable_scope('optimizer'):
                d_optimizer = tf.train.AdamOptimizer(learning_rate=D_LR, beta1=0.5)
                d_optimizer = tf.contrib.tpu.CrossShardOptimizer(d_optimizer)
            
                g_optimizer = tf.train.AdamOptimizer(learning_rate=G_LR, beta1=0.5)
                g_optimizer = tf.contrib.tpu.CrossShardOptimizer(g_optimizer)
         
                with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                    d_step = d_optimizer.minimize(d_loss_reduced, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                                                                     scope='Discriminator'))
                    g_step = g_optimizer.minimize(g_loss_reduced, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                                                                     scope='Generator'))
                    increment_step = tf.assign_add(tf.train.get_or_create_global_step(), 1)
                    joint_op = tf.group([d_step, g_step, increment_step])

                    a = tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=d_loss_reduced+g_loss_reduced, train_op=joint_op)
                    return a

        # EVAL #
        elif mode == tf.estimator.ModeKeys.EVAL:
            def _eval_metric_fn(d_loss, g_loss, d_real_labels, d_gen_lanels, d_real_logits, d_gen_logits):
                return {
                    'discriminator_loss': tf.metrics.mean(d_loss),
                    'generator_loss': tf.metrics.mean(g_loss),
                    'discriminator_real_accuracy': tf.metrics.accuracy(labels=d_real_labels, predictions=tf.math.round(tf.sigmoid(d_real_logits))),
                    'discriminator_gen_accuracy': tf.metrics.accuracy(labels=d_gen_lanels, predictions=tf.math.round(tf.sigmoid(d_gen_logits))),
                }

            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=d_loss_reduced + g_loss_reduced,
                                                   eval_metrics=(_eval_metric_fn, [d_loss, g_loss, 
                                                                                   tf.ones_like(labels), tf.zeros_like(labels),
                                                                                   d_on_data_logits, d_on_g_logits]))
    return model_fn

In [0]:
################################ ESTIMATORS ####################################

def make_estimators(model, only_cpu=False):
    model_fn = make_model_fn(model)
    
    config = tf.contrib.tpu.RunConfig(
        master=TF_MASTER,
        save_checkpoints_steps=EVAL_EPOCHS,
        save_checkpoints_secs=None,
        save_summary_steps=EVAL_EPOCHS,
        model_dir=MODEL_DIR,
        keep_checkpoint_max=3,
        tpu_config=tf.contrib.tpu.TPUConfig(iterations_per_loop=EVAL_EPOCHS))

    if not only_cpu:
        # TPU-based estimator used for TRAIN and EVAL
        est = tf.contrib.tpu.TPUEstimator(
            model_fn=model_fn,
            use_tpu=True,
            config=config,
            train_batch_size=BATCH_SIZE,
            eval_batch_size=BATCH_SIZE)
    else:
        est = None

    # CPU-based estimator used for PREDICT (generating images)
    cpu_est = tf.contrib.tpu.TPUEstimator(
        model_fn=model_fn,
        use_tpu=False,
        config=config,
        predict_batch_size=EXAMPLES)
    
    return est, cpu_est

In [0]:
################################# TRAINING #####################################

def train(est, cpu_est):
    current_step = estimator._load_global_step_from_checkpoint_dir(MODEL_DIR)
    print('Starting training')

    while current_step < EPOCHS:
        next_checkpoint = int(min(current_step + EVAL_EPOCHS, EPOCHS))
        est.train(input_fn=make_input_fn(), max_steps=next_checkpoint)
        current_step = next_checkpoint
        print('Finished training step %d' % current_step)

        # Evaluation
        metrics = est.evaluate(input_fn=make_input_fn(False), steps=1)
        print('Finished evaluating')
        print(metrics)

        # Render some generated images
        generated_iter = cpu_est.predict(input_fn=make_input_fn(False, one_batch=True))
        images = [np.concatenate([p['content_images'], p['style_images'], p['generated_images']], axis=1) for p in generated_iter]
        save_imgs(str(current_step), images)
        print('Finished generating images')

In [0]:
def do_experiment():
    setup()
    upload_credentials()
    model = Funit()
    est, cpu_est = make_estimators(model)
    train(est, cpu_est)

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

try:
    do_experiment()
except Exception as e:
    print (e)
    pass

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


W0819 12:27:16.589750 139671745042304 estimator.py:1984] Estimator's model_fn (<function make_model_fn.<locals>.model_fn at 0x7f07a372aa60>) includes params argument, but params are not passed to Estimator.
I0819 12:27:16.591437 139671745042304 estimator.py:209] Using config: {'_model_dir': 'gs://tputestingmnist/FUNIT/FUNIT_only_gen_11/', '_tf_random_seed': None, '_save_summary_steps': 1000, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0793dc3f28>, '_task_type': 'worker', '_task_i

Starting training


W0819 12:27:17.872776 139671745042304 deprecation.py:323] From <ipython-input-50-f6cca3d1f14d>:40: conv2d_transpose (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.


(128, 1, 1, 2048)


...

I0819 17:58:47.679443 139671745042304 estimator.py:1147] Done calling model_fn.
I0819 17:58:47.892158 139671745042304 monitored_session.py:240] Graph was finalized.
I0819 17:58:47.939489 139671745042304 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_gen_11/model.ckpt-47000
I0819 17:58:48.340764 139671745042304 session_manager.py:500] Running local_init_op.
I0819 17:58:48.362183 139671745042304 session_manager.py:502] Done running local_init_op.
I0819 17:58:48.734695 139671745042304 error_handling.py:96] prediction_loop marked as finished
I0819 17:58:48.736178 139671745042304 error_handling.py:96] prediction_loop marked as finished


Finished generating images


I0819 17:58:50.018962 139671745042304 estimator.py:1145] Calling model_fn.


(128, 1, 1, 2048)


I0819 17:58:58.221470 139671745042304 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0819 17:58:58.443117 139671745042304 estimator.py:1147] Done calling model_fn.
I0819 17:58:58.444517 139671745042304 tpu_estimator.py:499] TPU job name tpu_worker
I0819 17:58:59.200584 139671745042304 monitored_session.py:240] Graph was finalized.
I0819 17:58:59.365032 139671745042304 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_gen_11/model.ckpt-47000
I0819 17:59:05.534416 139671745042304 session_manager.py:500] Running local_init_op.
I0819 17:59:05.988817 139671745042304 session_manager.py:502] Done running local_init_op.
I0819 17:59:11.879301 139671745042304 basic_session_run_hooks.py:606] Saving checkpoints for 47000 into gs://tputestingmnist/FUNIT/FUNIT_only_gen_11/model.ckpt.
I0819 17:59:33.242277 139671745042304 util.py:98] Initialized dataset iterators in 0 seconds
I0819 17:59:33.243590 139671745042304 session_support.py:332] Installing graceful s